In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,35256
2,Huelva,Confirmados PDIA 14 días,1498
3,Huelva,Tasa PDIA 14 días,"291,9110626108307"
4,Huelva,Confirmados PDIA 7 días,864
5,Huelva,Total Confirmados,35447
6,Huelva,Curados,32644
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  35256.0
/tmp/ipykernel_177775/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9743.0
/tmp/ipykernel_177775/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_177775/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_177775/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_177775/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 427 personas en los últimos 7 días 

Un positivo PCR cada 288 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,35256.0,864.0,1498.0,513170.0,168.365259,291.911063,209.0
Huelva-Costa,20547.0,616.0,1061.0,289548.0,212.745382,366.433199,157.0
Huelva (capital),9743.0,337.0,500.0,143837.0,234.292984,347.615704,84.0
Condado-Campiña,11300.0,186.0,349.0,156231.0,119.054477,223.387164,39.0
Lepe,2319.0,57.0,116.0,27880.0,204.447633,416.068867,21.0
Isla Cristina,2411.0,70.0,179.0,21393.0,327.209835,836.722292,17.0
Sierra de Huelva-Andévalo Central,3038.0,50.0,72.0,67391.0,74.193883,106.839192,15.0
Ayamonte,1460.0,41.0,81.0,21104.0,194.275967,383.813495,12.0
Moguer,1499.0,47.0,69.0,21867.0,214.935748,315.543970,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Niebla,306.0,20.0,57.0,4116.0,485.908649,1384.839650,1.0
Isla Cristina,2411.0,70.0,179.0,21393.0,327.209835,836.722292,17.0
Lepe,2319.0,57.0,116.0,27880.0,204.447633,416.068867,21.0
Zarza-Perrunal (La),34.0,3.0,5.0,1253.0,239.425379,399.042298,0.0
Ayamonte,1460.0,41.0,81.0,21104.0,194.275967,383.813495,12.0
Huelva-Costa,20547.0,616.0,1061.0,289548.0,212.745382,366.433199,157.0
Palos de la Frontera,779.0,32.0,43.0,11742.0,272.525975,366.206779,8.0
Huelva (capital),9743.0,337.0,500.0,143837.0,234.292984,347.615704,84.0
Aljaraque,1257.0,47.0,73.0,21474.0,218.869330,339.945981,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,208.0,1.0,12.0,3933.0,25.425884,305.110603,0.0,0.083333
Rosal de la Frontera,37.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Bollullos Par del Condado,1064.0,5.0,15.0,14387.0,34.753597,104.260791,2.0,0.333333
Niebla,306.0,20.0,57.0,4116.0,485.908649,1384.839650,1.0,0.350877
Lucena del Puerto,282.0,4.0,11.0,3261.0,122.661760,337.319841,0.0,0.363636
Almonte,1960.0,10.0,26.0,24507.0,40.804668,106.092137,2.0,0.384615
Gibraleón,693.0,12.0,31.0,12737.0,94.213708,243.385413,3.0,0.387097
Isla Cristina,2411.0,70.0,179.0,21393.0,327.209835,836.722292,17.0,0.391061
Lepe,2319.0,57.0,116.0,27880.0,204.447633,416.068867,21.0,0.491379
